## Extract and organize RNAseq data of CMV genes

In [8]:
import os
import pandas as pd

#### General parameters

In [9]:
gtf_f = "/Users/erankotler/Google Drive/workspace/CMV/Data/RNAseq/CMV.gtf"
CMV_gene_ID_conv_f = "/Users/erankotler/Google Drive/workspace/CMV/Data/Gene_ID_conversions/CMV_Conversion.csv" 
data_d = "/Users/erankotler/Google Drive/workspace/CMV/Data/RNAseq"
deseq_f = os.path.join(data_d,"de_stats.csv") # DEseq stats file
norm_counts_f = os.path.join(data_d, "countsMatrix_normalized.csv") # Normalized counts file

### Create gene ID conversion file for CMV genome

#### Load gtf file and extract gene names for conversion table
(conversion from https://www.ncbi.nlm.nih.gov/nuccore/157779983?report=genbank gff3 to gtf was performed by Gil Hornung, see readme.txt on cluster under CMV/RNAseq/INCPM/delivery)

In [10]:
gtf = pd.read_csv(gtf_f, header=None, sep='\t')
conv_tbl = pd.DataFrame()

conv_tbl["Gene_name"] = gtf[8].apply(lambda s:s.split("gene_name")[-1][:-1][2:-1])
conv_tbl["Gene_id"] = gtf[8].apply(lambda s:s.split("gene_id")[-1].split(";")[0][2:-1])
conv_tbl["Transcript_id"] = gtf[8].apply(lambda s:s.split("transcript_id")[-1].split(";")[0][2:-1])
conv_tbl = conv_tbl.drop_duplicates() # Dropping multiple transcripts of same gene

#### Save conversion table to csv

In [11]:
conv_tbl.to_csv(CMV_gene_ID_conv_f)

### Create differetial expression files for CMV genes (from DEseq output file)

In [12]:
# Load deseq data and gene name conversion table
conv = pd.read_csv(CMV_gene_ID_conv_f, index_col=2).drop(["Transcript_id","Unnamed: 0"], axis=1)
deseq = pd.read_csv(deseq_f, index_col=0) # Load DESeq statistics

In [13]:
comparisons = list(deseq.index.unique()) # List of all 6 pairwise comparisons 

In [14]:
# Merge datasets, leaving only CMV genes with known gene name
deseq_known_IDs = pd.merge(left = deseq, right=conv, left_on="Gene", right_index=True, how='inner')
deseq_known_IDs["Gene"] = deseq_known_IDs["Gene_name"]
deseq_known_IDs.drop("Gene_name", axis=1, inplace=True)

In [15]:
## Save files to csv:
# United de data file for CMV genes:
deseq_known_IDs.to_csv(os.path.join(data_d,"de_stats_CMV_genes.csv"))

# Separate file for each of the 6 pariwise comparisons:
for c in comparisons:
    d = deseq_known_IDs.loc[c, :]
    d.index = d["Gene"]
    d = d.drop("Gene", axis=1)
    d.to_csv(os.path.join(data_d,"{comp}_de_stats_CMV_genes.csv".format(comp=c)))

### Create normalized counts file for CMV genes

In [16]:
# Load deSeq normalized read counts data
norm_counts = pd.read_csv(norm_counts_f, index_col=0)

In [17]:
# Merge datasets, leaving only CMV genes with known gene name
counts_known_IDs = pd.merge(left = norm_counts, right=conv, left_index=True, right_index=True, how='inner')
counts_known_IDs["Gene"] = counts_known_IDs["Gene_name"]
counts_known_IDs.drop("Gene_name", axis=1, inplace=True)

In [18]:
# Reorder columns:
cols_order = ['Gene',
              '1_Luc_Mock',
              '2_Luc_Mock',
              '3_Luc_Mock',
              '4_Luc_CMV',
              '5_Luc_CMV',
              '6_Luc_CMV',
              '7_IE1wt_CMV',
              '8_IE1wt_CMV',
              '9_IE1wt_CMV',
              '10_IE1dCTD_CMV',
              '11_IE1dCTD_CMV',
              '12_IE1dCTD_CMV']

In [19]:
counts_known_IDs = counts_known_IDs[cols_order]

In [20]:
## Save CMV counts data to csv:
counts_known_IDs.to_csv(os.path.join(data_d,"countsMatrix_normalized_CMV_genes.csv"))

### Create normalized counts file for human genes as well

In [21]:
human_gene_ID_conv_f = "/Users/erankotler/Google Drive/workspace/CMV/Data/Gene_ID_conversions/BioMart_Conversion.csv" 
human_conv = pd.read_csv(human_gene_ID_conv_f, index_col=2).drop("Transcript stable ID", axis=1)
human_conv = human_conv.drop_duplicates()

In [22]:
# Merge datasets, leaving only human genes with known gene name
human_counts_known_IDs = pd.merge(left = norm_counts, right=human_conv, left_index=True, right_index=True, how='inner')
human_counts_known_IDs["Gene"] = human_counts_known_IDs["Gene name"]
human_counts_known_IDs.drop("Gene name", axis=1, inplace=True)

In [23]:
human_counts_known_IDs = human_counts_known_IDs[cols_order]

In [24]:
## Save human counts data to csv:
human_counts_known_IDs.to_csv(os.path.join(data_d,"countsMatrix_normalized_human_genes.csv"))

### Create gene-name converted TPM file for CMV genes

In [36]:
tpm_f = os.path.join(data_d,"TPMs_kallisto.tsv")
tpm = pd.read_csv(tpm_f, sep=' ', index_col=0)
tpm.index.name=None

In [37]:
# Merge datasets, leaving only human genes with known gene name
cmv_tpm_known_IDs = pd.merge(left = tpm, right=conv, left_index=True, right_index=True, how='inner')
cmv_tpm_known_IDs["Gene"] = cmv_tpm_known_IDs["Gene_name"]
cmv_tpm_known_IDs.drop("Gene_name", axis=1, inplace=True)

In [38]:
cmv_tpm_known_IDs = cmv_tpm_known_IDs[cols_order]

In [39]:
## Save human TPM data to csv:
cmv_tpm_known_IDs.to_csv(os.path.join(data_d,"TPMs_kallisto_CMV_genes.csv"))

### Create gene-name converted TPM file for human data

In [26]:
# Merge datasets, leaving only human genes with known gene name
human_tpm_known_IDs = pd.merge(left = tpm, right=human_conv, left_index=True, right_index=True, how='inner')
human_tpm_known_IDs["Gene"] = human_tpm_known_IDs["Gene name"]
human_tpm_known_IDs.drop("Gene name", axis=1, inplace=True)

In [27]:
human_tpm_known_IDs = human_tpm_known_IDs[cols_order]

In [28]:
## Save human TPM data to csv:
human_tpm_known_IDs.to_csv(os.path.join(data_d,"TPMs_kallisto_human_genes_convertedNames.csv"))